# Next 16-days Temp, Rainfall, humidity, wind, alerts

In [15]:
district = "LUCKNOW"

In [21]:
import os
import requests
import numpy as np
import pandas as pd

OWM_API_KEY = "b93651e4ede9b7679f28a549efcae210"  # replace with your OpenWeather key

def get_coords(district, limit=5):
    """
    Returns (lat, lon) for a district using OpenWeather's direct geocoding API.
    Uses only the district name in the query.
    """
    url = "http://api.openweathermap.org/geo/1.0/direct"
    params = {
        "q": district,   # ✅ only district name
        "limit": limit,
        "appid": OWM_API_KEY,
    }
    resp = requests.get(url, params=params, timeout=15)
    resp.raise_for_status()
    results = resp.json()

    if not results:
        raise ValueError(f"No coordinates found for '{district}'")

    return results[0]["lat"], results[0]["lon"]


lat, lon = get_coords(district)
print({"lat": lat, "lon": lon})


# === Open-Meteo API Calls ===
url1 = (
    "https://api.open-meteo.com/v1/forecast"
    f"?latitude={lat}&longitude={lon}"
    "&daily=temperature_2m_max,temperature_2m_min,precipitation_sum,wind_speed_10m_max,wind_gusts_10m_max"
    "&timezone=Asia/Kolkata"
    "&forecast_days=16"
)

url2 = (
    "https://api.open-meteo.com/v1/forecast"
    f"?latitude={lat}&longitude={lon}"
    "&hourly=relative_humidity_2m"
    "&timezone=Asia/Kolkata"
    "&forecast_days=16"
)

resp = requests.get(url1, timeout=15)
resp.raise_for_status()
data = resp.json()

resp2 = requests.get(url2, timeout=15)
resp2.raise_for_status()
data2 = resp2.json()

# === Validation for required fields ===
required_daily_keys = ["time", "temperature_2m_max", "temperature_2m_min", "precipitation_sum", "wind_speed_10m_max", "wind_gusts_10m_max"]
for key in required_daily_keys:
    if key not in data.get("daily", {}):
        raise KeyError(f"Missing key '{key}' in Open-Meteo daily response")

if "time" not in data2.get("hourly", {}) or "relative_humidity_2m" not in data2.get("hourly", {}):
    raise KeyError("Missing humidity data in Open-Meteo hourly response")


# === Build DataFrames ===
df = pd.DataFrame({
    'date': data['daily']['time'],
    'temp_max': data['daily']['temperature_2m_max'],
    'temp_min': data['daily']['temperature_2m_min'],
    'precip_mm': data['daily']['precipitation_sum'],
    'wind_speed': data['daily']['wind_speed_10m_max'],
    'wind_gusts': data['daily']['wind_gusts_10m_max'],
})

df2 = pd.DataFrame({
    'date': data2['hourly']['time'],
    'humidity': data2['hourly']['relative_humidity_2m']
})

# Convert hourly → daily avg humidity
df2["date"] = pd.to_datetime(df2["date"]).dt.strftime("%Y-%m-%d")
humidity_daily = (
    df2.groupby("date", as_index=False)["humidity"].mean()
    .rename(columns={"humidity": "humidity_mean"})
)

df = df.merge(
    humidity_daily.rename(columns={"humidity_mean": "humidity"}),
    on="date",
    how="left"
)

# === Averages (16-day) ===
avg_prec_16d = float(df["precip_mm"].mean())
avg_temp_max_16d = float(df["temp_max"].mean())
avg_temp_min_16d = float(df["temp_min"].mean())
avg_humidity_16d = float(df["humidity"].mean())
avg_wind_s_16d = float(df["wind_speed"].mean())
avg_wind_g_16d = float(df["wind_gusts"].mean())

print(f"Avg precipitation (16 days): {avg_prec_16d:.2f} mm")
print(f"Avg temperature Min (16 days): {avg_temp_min_16d:.2f} °C")
print(f"Avg temperature Max (16 days): {avg_temp_max_16d:.2f} °C")
print(f"Avg humidity (16 days): {avg_humidity_16d:.2f} %")
print(f"Avg wind speed (16 days): {avg_wind_s_16d:.2f} m/s")
print(f"Avg wind gusts (16 days): {avg_wind_g_16d:.2f} m/s")

# === Final Input Dict ===
try:
    inputt
except NameError:
    inputt = {}

inputt.update({
    "temperature_max": avg_temp_max_16d,
    "temperature_min": avg_temp_min_16d,
    "humidity": avg_humidity_16d,
    "rainfall": avg_prec_16d,
    "wind_speed": avg_wind_s_16d,
    "wind_gusts": avg_wind_g_16d
})

# Example array for ML model input
val_arr = np.array(list(inputt.values()), dtype=float)
df["temp"] = (df["temp_min"] + df["temp_max"]) / 2



{'lat': 26.8381, 'lon': 80.9346001}
Avg precipitation (16 days): 4.74 mm
Avg temperature Min (16 days): 26.69 °C
Avg temperature Max (16 days): 32.60 °C
Avg humidity (16 days): 75.34 %
Avg wind speed (16 days): 13.64 m/s
Avg wind gusts (16 days): 30.11 m/s


In [22]:
df

,date,temp_max,temp_min,precip_mm,wind_speed,wind_gusts,humidity,temp
0,2025-08-21,31.6,26.5,22.9,10.0,27.0,85.375000,29.05
1,2025-08-22,30.4,25.6,7.7,8.8,25.6,87.291667,28.00
2,2025-08-23,30.3,25.6,8.6,13.9,36.7,87.625000,27.95
3,2025-08-24,28.5,25.3,6.0,8.5,24.1,88.333333,26.90
4,2025-08-25,29.2,25.5,14.7,7.8,26.3,90.958333,27.35
5,2025-08-26,31.2,25.5,8.4,9.2,33.5,84.666667,28.35
6,2025-08-27,33.9,26.1,0.0,8.4,27.0,78.916667,30.00
7,2025-08-28,33.4,26.3,2.4,8.0,25.6,79.333333,29.85
8,2025-08-29,32.3,26.7,2.1,31.7,49.7,76.416667,29.50
9,2025-08-30,33.1,25.7,0.0,19.7,46.1,69.541667,29.40


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# 2. Derived Weather Features

# Daily Temperature Range
df["dtr"] = df["tmax"] - df["tmin"]

# Heat Index (simplified)
df["heat_index"] = df["temp"] + 0.33*df["humidity"]/100*6.105*np.exp(17.27*df["temp"]/(237.7+df["temp"])) - 4

# Wind Chill (valid when temp < 10°C)
df["wind_chill"] = np.where(
    (df["temp"] < 10) & (df["wind_speed"] > 4.8),
    13.12 + 0.6215*df["temp"] - 11.37*(df["wind_speed"]**0.16) + 0.3965*df["temp"]*(df["wind_speed"]**0.16),
    np.nan
)

# Wind Risk Index
df["wind_risk"] = df["wind_gusts"] / (df["wind_speed"] + 1)

# Approx Evapotranspiration (ET0) using Hargreaves (simplified)
df["et0"] = 0.0023 * (df["tmax"] - df["tmin"])**0.5 * (df["temp"] + 17.8)

# Aridity Index (precipitation vs ET)
df["aridity_index"] = df["precip_mm"] / (df["et0"] + 0.01)

# ==============================
# 3. Crop Stress Flags
# ==============================

def classify_flags(row):
    flags = {}

    # Heat Stress
    if row["tmax"] > 38:
        flags["heat"] = "🔴 Red (Heat stress)"
    elif row["tmax"] > 32:
        flags["heat"] = "🟡 Yellow (Mild stress)"
    else:
        flags["heat"] = "🟢 Green (Safe)"

    # Cold Stress
    if row["tmin"] < 5:
        flags["cold"] = "🔴 Red (Frost risk)"
    elif row["tmin"] < 10:
        flags["cold"] = "🟡 Yellow (Chill stress)"
    else:
        flags["cold"] = "🟢 Green (Safe)"

    # Drought/Irrigation Need
    if row["aridity_index"] < 0.5:
        flags["water"] = "🔴 Red (Irrigation needed)"
    elif row["aridity_index"] < 1:
        flags["water"] = "🟡 Yellow (Monitor)"
    else:
        flags["water"] = "🟢 Green (Sufficient)"

    # Wind Damage
    if row["wind_gusts"] > 60:
        flags["wind"] = "🔴 Red (Lodging risk)"
    elif row["wind_gusts"] > 40:
        flags["wind"] = "🟡 Yellow (Caution)"
    else:
        flags["wind"] = "🟢 Green (Safe)"

    return flags

df["flags"] = df.apply(classify_flags, axis=1)

# ==============================
# 4. Crop Recommendations
# ==============================

def crop_recommendation(row):
    recs = []

    # Example logic — you can expand
    if row["heat_index"] > 35:
        recs.append("Avoid transplanting rice/wheat, risk of heat stress.")
    if row["aridity_index"] < 0.7:
        recs.append("Schedule irrigation (maize, rice sensitive).")
    if row["wind_gusts"] > 50:
        recs.append("Support tall crops (sugarcane, maize) to prevent lodging.")
    if row["tmin"] < 8:
        recs.append("Protect seedlings from cold (vegetables, potato).")

    if not recs:
        recs.append("Conditions favorable — proceed with normal farming.")
    
    return " | ".join(recs)

df["recommendations"] = df.apply(crop_recommendation, axis=1)

# ==============================
# 5. Visualization Example
# ==============================

plt.figure(figsize=(12,6))
plt.plot(df["date"], df["tmax"], label="Tmax", color="red")
plt.plot(df["date"], df["tmin"], label="Tmin", color="blue")
plt.bar(df["date"], df["precip_mm"], label="Precip (mm)", alpha=0.3)
plt.legend()
plt.title("Daily Weather Forecast with Tmin/Tmax & Rain")
plt.xlabel("Date")
plt.ylabel("°C / mm")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# ==============================
# 6. Save Final Output
# ==============================
df.to_csv("weather_agri_pipeline.csv", index=False)

# Preview
print(df[["date","tmin","tmax","precip_mm","et0","aridity_index","flags","recommendations"]].head())
